# Processing LAPD calls for service: 2010-2020

### Load python tools

In [1]:
import json
import pandas as pd
import matplotlib
import numpy as np
import altair as alt
from altair import datum
import altair_latimes as lat
pd.options.display.max_columns = 50
pd.options.display.max_rows = 500
pd.set_option('display.max_colwidth', None)
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
alt.renderers.enable('default')
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
pd.option_context('display.precision', 2)
def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'red' if val < 0 else 'black'
    return 'color: %s' % color

### Download latest data from all sources

In [3]:
# %run 'download_calls.ipynb'

## Los Angeles Police

In [4]:
df = pd.read_csv('/Users/mhustiles/data/data/LA/calls/la/los_angeles.csv', low_memory=False)

In [5]:
df.head()

,incident_number,reporting_district,area_occurred,dispatch_date,dispatch_time,call_type_code,call_type_description,date,time
0,PD20012600002716,Rampart,0245,01/26/2020,14:37:04,211S,SUSP,2020-01-26,14:37:04
1,PD20011400005678,Outside,NaN,01/14/2020,23:26:20,006,CODE 6,2020-01-14,23:26:20
2,PD20011100005314,Outside,NaN,01/11/2020,23:32:40,006,CODE 6,2020-01-11,23:32:40
3,PD20010100005220,Southwest,0328,01/01/2020,22:34:29,245SN,SUSP NOW,2020-01-01,22:34:29
4,PD20010500000660,Hollywood,0636,01/05/2020,04:04:42,907PA2,POSS AMB O/D,2020-01-05,04:04:42


In [6]:
df['date'] = pd.to_datetime(df['dispatch_date'], format='%m/%d/%Y')
df['monthday'] = df['date'].dt.strftime('%m/%d')
df['year'] = df['date'].dt.strftime('%Y')

In [7]:
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['date'] = df['date'].dt.date

In [8]:
df = df.drop(['dispatch_time', 'dispatch_date'], axis=1)

In [9]:
los_angeles = df.copy()

In [10]:
los_angeles['outside_yn'] = los_angeles['reporting_district'] == 'Outside'

In [11]:
los_angeles.head()

,incident_number,reporting_district,area_occurred,call_type_code,call_type_description,date,time,monthday,year,month,day,outside_yn
0,PD20012600002716,Rampart,0245,211S,SUSP,2020-01-26,14:37:04,01/26,2020,1,26,False
1,PD20011400005678,Outside,NaN,006,CODE 6,2020-01-14,23:26:20,01/14,2020,1,14,True
2,PD20011100005314,Outside,NaN,006,CODE 6,2020-01-11,23:32:40,01/11,2020,1,11,True
3,PD20010100005220,Southwest,0328,245SN,SUSP NOW,2020-01-01,22:34:29,01/01,2020,1,1,False
4,PD20010500000660,Hollywood,0636,907PA2,POSS AMB O/D,2020-01-05,04:04:42,01/05,2020,1,5,False


---

### What were call volumes like each day this year? 

In [12]:
this_year = los_angeles[los_angeles['year'] == '2020'].groupby(['monthday']).agg('size').reset_index(name='calls').drop([59], axis=0)

### When were the protests? 

In [17]:
this_year_before = this_year[(this_year['monthday'] < '05/30')]
protest_period = pd.DataFrame(this_year[(this_year['monthday'] > '05/27') & (this_year['monthday'] < '06/10')])

In [18]:
los_angeles_protests_during = pd.DataFrame(los_angeles[(los_angeles['monthday'] > '05/27') & (los_angeles['monthday'] < '06/10')])
los_angeles_protests_before = pd.DataFrame(los_angeles[los_angeles['monthday'] < '05/30'])

### Slice out dataframes for all codes that aren't 'Code 6' and are outside LAPD jurisdiction

In [19]:
no_code6 = los_angeles[los_angeles['call_type_code'] != '006']
outside = los_angeles[los_angeles['reporting_district'] == 'Outside']

In [20]:
this_year_no_code6_all = no_code6[no_code6['year'] == '2020'].groupby(['monthday']).agg('size').reset_index(name='calls')

In [21]:
protest_period_no_code_6 = pd.DataFrame(this_year_no_code6_all[(this_year_no_code6_all['monthday'] > '05/27') & (this_year['monthday'] < '06/10')])
protest_period['mean_no_code6'] = 4000
protest_period['mean_before'] = this_year_before.calls.max()

### What was the percentage change in the average number of calls before and during the protest period?

In [22]:
(protest_period.calls.mean() - this_year_before.calls.mean())/this_year_before.calls.mean()

-0.4233551024581533

### Chart all the daily calls as a time series

In [23]:
chart = alt.Chart((this_year))\
    .mark_area(size=3,opacity=.6, color='#B32F2E', interpolate='monotone')\
    .encode(
    x=alt.X('monthday:T', title=' ', axis=alt.Axis(grid=False, tickCount=6, format='%B')),
    y=alt.Y('calls:Q', stack=None, title=' ', axis=alt.Axis(gridWidth=.6,\
     gridColor='#dddddd',offset=6,tickSize=0,domainOpacity=0,tickCount=4, format=''),\
           scale=alt.Scale(domain=(0, 6000)))
).properties(width=630, height=400,
     title='All service calls to LAPD: 2020'
)

rule = alt.Chart(this_year).mark_rule(color='#666', size=1).encode(
    y='mean(calls):Q',
)

text = rule.mark_text(
    color="#000000", font='Benton Gothic', fontSize=12,
    align='center',
    baseline='middle',
    dx=112,
    dy=12,
    fontWeight='bold',
).encode(
    text=alt.Text('label:N', format="")).transform_calculate(label='"Average: 4,900"')

text_protest = rule.mark_text(
    color="#000000", font='Benton Gothic', fontSize=12,
    align='center',
    baseline='middle',
    dx=252,
    dy=-130,
    fontWeight='bold',
).encode(
    text=alt.Text('label:N', format="")).transform_calculate(label='"Protests"')

protest_annotation = alt.Chart(protest_period).mark_area(color='#999', opacity=.3).encode(
    y='mean_before',
    x='monthday:T'
)

(protest_annotation + chart + text + text_protest + rule).configure_view(strokeOpacity=0).configure_legend(
    orient='top',
    symbolType='stroke'
).configure_legend(symbolType='square')

alt.LayerChart(...)

---

### What are these 'Code 6' calls to LAPD?

'Code 6' is an officer-initiated call. When they observe something, they go 'Code 6' and give their location. In 2020, these were is a significant portion (45%) of the daily volume. 

In [24]:
los_angeles[los_angeles['year'] == '2020'].call_type_description.value_counts('normalize').head()

CODE 6          0.448007
TRAFFIC STOP    0.062945
MAN             0.042802
GRP             0.026967
SUSP NOW        0.023138
Name: call_type_description, dtype: float64

### Without those 'Code 6' calls, traffic stops are the largest group

In [25]:
los_angeles[(los_angeles['year'] == '2020') & (los_angeles['call_type_description'] != 'CODE 6')].call_type_description.value_counts('normalize').head()

TRAFFIC STOP     0.114032
MAN              0.077541
GRP              0.048855
SUSP NOW         0.041917
TRESPASS SUSP    0.039465
Name: call_type_description, dtype: float64

### What's the share of calls per day this year from outside LAPD jurisdiction, by type?

In [26]:
this_year_outside = outside[outside['year'] == '2020'].groupby(['monthday', 'call_type_description']).agg('size').reset_index(name='calls')

In [27]:
this_year_outside.call_type_description.value_counts('normalize').head()

CODE 6          0.210526
TRAFFIC STOP    0.210526
BACK-UP         0.162791
INVEST          0.085679
HELP            0.075887
Name: call_type_description, dtype: float64

In [28]:
los_angeles.outside_yn.value_counts('normalize').head()

False    0.893508
True     0.106492
Name: outside_yn, dtype: float64

### Various frames for charting

In [29]:
this_year_no_code6_all = no_code6[no_code6['year'] == '2020'].groupby(['monthday']).agg('size').reset_index(name='calls')

In [30]:
this_year_no_code6 = no_code6[no_code6['year'] == '2020'].groupby(['monthday', 'reporting_district']).agg('size').reset_index(name='calls')

In [31]:
this_year_w_code6 = los_angeles[los_angeles['year'] == '2020'].groupby(['monthday', 'reporting_district']).agg('size').reset_index(name='calls')

In [32]:
this_year_traffic_stop = los_angeles[(los_angeles['year'] == '2020') \
                                     & (los_angeles['call_type_description'] == 'TRAFFIC STOP')].groupby(['monthday', 'reporting_district']).agg('size').reset_index(name='calls')

In [33]:
this_year_no_code6_all.calls.mean()

2729.389534883721

### How does the 'all calls' chart change without 'Code 6' incidents?

In [34]:
chart = alt.Chart((this_year_no_code6_all))\
    .mark_area(size=3,opacity=.6, color='#B32F2E', interpolate='monotone')\
    .encode(
    x=alt.X('monthday:T', title=' ', axis=alt.Axis(grid=False, tickCount=6, format='%B')),
    y=alt.Y('calls:Q', stack=None, title=' ', axis=alt.Axis(gridWidth=.6,\
     gridColor='#dddddd',offset=6,tickSize=0,domainOpacity=0,tickCount=4, format=''),\
           scale=alt.Scale(domain=(0, 4000)))
).properties(width=630, height=400,
     title='All service calls (no code 6) to LAPD: 2020'
)

rule = alt.Chart(this_year_no_code6_all).mark_rule(color='#666', size=1).encode(
    y='mean(calls):Q',
)

text = rule.mark_text(
    color="#000000", font='Benton Gothic', fontSize=12,
    align='center',
    baseline='middle',
    dx=112,
    dy=12,
    fontWeight='bold',
).encode(
    text=alt.Text('label:N', format="")).transform_calculate(label='"Average: 2,700"')

text_protest = rule.mark_text(
    color="#000000", font='Benton Gothic', fontSize=12,
    align='center',
    baseline='middle',
    dx=252,
    dy=-110,
    fontWeight='bold',
).encode(
    text=alt.Text('label:N', format="")).transform_calculate(label='"Protests"')

protest_annotation = alt.Chart(protest_period).mark_area(color='#999', opacity=.3).encode(
    y='mean_no_code6',
    x='monthday:T'
)

(protest_annotation + chart + text + text_protest + rule).configure_view(strokeOpacity=0).configure_legend(
    orient='top',
    symbolType='stroke'
).configure_legend(symbolType='square')

alt.LayerChart(...)

### Which types of calls dropped the most during the protest period?

In [35]:
calls_all_year = los_angeles[los_angeles['year'] == '2020'].groupby(['date','call_type_description']).agg('size').reset_index(name='calls')

In [36]:
calls_during_protests = los_angeles_protests_during[los_angeles_protests_during['year'] == '2020']\
    .groupby(['date','call_type_description']).agg('size').reset_index(name='calls')

In [37]:
calls_before_protests = los_angeles_protests_before[los_angeles_protests_before['year'] == '2020'].groupby(['date','call_type_description']).agg('size').reset_index(name='calls')

In [38]:
types_during = calls_during_protests.groupby(['call_type_description']).agg('mean').reset_index()

In [39]:
types_before = calls_before_protests.groupby(['call_type_description']).agg('mean').reset_index()

In [40]:
types_during.sort_values(by='calls', ascending=False).head(5)

,call_type_description,calls
92,CODE 6,1183.615385
308,SUSP NOW,157.000000
91,CODE 30 RINGER,125.538462
193,MAN,98.538462
112,DOM VIOL,82.846154


In [41]:
compare_calls = pd.DataFrame(types_before.merge(types_during, on='call_type_description').rename(columns={'calls_x':'avg_daily_before', 'calls_y':'avg_daily_after'}))

In [42]:
compare_calls['pct_change'] = ((compare_calls['avg_daily_after'] - compare_calls['avg_daily_before'])/compare_calls['avg_daily_before']).round(2)

In [43]:
compare_calls['raw_change'] = compare_calls['avg_daily_after'] - compare_calls['avg_daily_before']

### Which types of calls saw the largest raw change during the protests?

In [44]:
compare_calls.sort_values(by='raw_change', ascending=True).head()

,call_type_description,avg_daily_before,avg_daily_after,pct_change,raw_change
92,CODE 6,2349.020000,1183.615385,-0.50,-1165.404615
318,TRAFFIC STOP,340.753333,66.307692,-0.81,-274.445641
193,MAN,222.446667,98.538462,-0.56,-123.908205
145,GRP,143.413333,33.076923,-0.77,-110.336410
319,TRESPASS SUSP,116.086667,24.076923,-0.79,-92.009744


### Which types of calls saw the largest % change during the protests?

In [45]:
compare_calls.sort_values(by='pct_change', ascending=True).head()

,call_type_description,avg_daily_before,avg_daily_after,pct_change,raw_change
228,PARTY,78.173333,9.250000,-0.88,-68.923333
94,COMPLAINT,27.133333,3.700000,-0.86,-23.433333
110,DISPUTE PUBLIC ORDER,29.000000,4.666667,-0.84,-24.333333
282,RADIO,74.806667,14.000000,-0.81,-60.806667
318,TRAFFIC STOP,340.753333,66.307692,-0.81,-274.445641


---

### Inside vs. outside?

In [46]:
los_angeles_inside = los_angeles[(los_angeles['reporting_district'] != 'Outside') & (los_angeles['year'] == '2020') ]

In [47]:
calls_all_year_inside = los_angeles_inside[los_angeles_inside['year'] == '2020'].groupby(['date','call_type_description']).agg('size').reset_index(name='calls')

In [48]:
types_inside = calls_all_year_inside.groupby(['call_type_description']).agg({'mean', 'sum'}).reset_index()

In [ ]:
los_angeles.head()

---

In [ ]:
previous_year_fireworks = los_angeles[(los_angeles['month'] < 6) & (los_angeles['call_type_description'] == 'FIREWORKS')]\
    .groupby(['year']).agg('size').reset_index(name='calls')

In [ ]:
previous_year_fireworks.to_csv('output/fireworks.csv')

In [ ]:
previous_year_fireworks.calls.max()

In [ ]:
this_year_fireworks = los_angeles[(los_angeles['year'] == '2020') & (los_angeles['call_type_description'] == 'FIREWORKS')].groupby(['monthday', 'reporting_district']).agg('size').reset_index(name='calls')

In [ ]:
chart = alt.Chart((this_year_fireworks))\
    .mark_bar(size=1,opacity=1, color='#B32F2E')\
    .encode(
    x=alt.X('monthday:T', title=' ', axis=alt.Axis(grid=False, tickCount=3, format='%B')),
    y=alt.Y('calls:Q', stack=None, title=' ', axis=alt.Axis(gridWidth=.6,\
     gridColor='#dddddd',offset=6,tickSize=0,domainOpacity=0,tickCount=4, format=''),\
           scale=alt.Scale(domain=(0, 30))),
    facet=alt.Facet('reporting_district:O', columns=7, title='LAPD reporting district')
).properties(width=100, height=100,
     title='All fireworks calls to LAPD: 2020'
)

chart.configure_view(strokeOpacity=0).configure_legend(
    orient='top',
    symbolType='stroke'
).configure_legend(symbolType='square')

In [ ]:
chart = alt.Chart((this_year_traffic_stop))\
    .mark_bar(size=1,opacity=1, color='#B32F2E')\
    .encode(
    x=alt.X('monthday:T', title=' ', axis=alt.Axis(grid=False, tickCount=6, format='%B')),
    y=alt.Y('calls:Q', stack=None, title=' ', axis=alt.Axis(gridWidth=.6,\
     gridColor='#dddddd',offset=6,tickSize=0,domainOpacity=0,tickCount=4, format=''),\
           scale=alt.Scale(domain=(0, 500))),
    facet=alt.Facet('reporting_district:O', columns=6, title='LAPD reporting district')
).properties(width=100, height=100,
     title='All traffic stops handled by LAPD: 2020'
)

chart.configure_view(strokeOpacity=0).configure_legend(
    orient='top',
    symbolType='stroke'
).configure_legend(symbolType='square')

### Which cases are related to fireworks?

In [ ]:
compare_calls[compare_calls['call_type_description'] == 'FIREWORKS'].sort_values(by='pct_change', ascending=True).head(30)

In [57]:
fireworks = pd.DataFrame(df[df['call_type_code'] == '507F'])

In [58]:
daily = fireworks[fireworks['year'] != '2020'].groupby(['monthday']).agg('size').reset_index(name='average')

In [59]:
daily['average'] = daily['average']/10

In [60]:
thisyear = fireworks[fireworks['year'] == '2020'].groupby(['monthday']).agg('size').reset_index(name='average')

In [61]:
fireworks_trend = daily.merge(thisyear, on=['monthday'], how='left')

In [62]:
fireworks_trend.rename(columns={'average_x':'Average 2010-2019', 'average_y':'ThisYear'}, inplace=True)

In [63]:
fireworks_trend.ThisYear = fireworks_trend.ThisYear.ffill()

In [64]:
fireworks_chart = pd.DataFrame(fireworks_trend[fireworks_trend['monthday'] < '06/20'])

In [65]:
fireworks_melt = pd.melt(fireworks_chart, id_vars=['monthday'], value_vars=['Average 2010-2019', 'ThisYear'],
        var_name='time', value_name='calls')

In [66]:
fireworks_melt.head()

,monthday,time,calls
0,01/01,Average 2010-2019,53.1
1,01/02,Average 2010-2019,6.1
2,01/03,Average 2010-2019,2.2
3,01/04,Average 2010-2019,2.9
4,01/05,Average 2010-2019,2.9


In [108]:
chart = alt.Chart((fireworks_melt))\
    .mark_bar(size=4,opacity=0.4)\
    .encode(
    x=alt.X('monthday:T', title=' ', axis=alt.Axis(grid=False, tickCount=6, format='%B')),
    y=alt.Y('calls:Q', stack=None, title=' ', axis=alt.Axis(gridWidth=.6,\
     gridColor='#dddddd',offset=6,tickSize=0,domainOpacity=0,tickCount=4, format=''),\
           scale=alt.Scale(domain=(0, 200))),
    color=alt.Color("time:N", title=' ')
).properties(width=630, height=400,
     title='Fireworks calls to LAPD: 2010-2019'
)

(chart).configure_view(strokeOpacity=0).configure_legend(
    orient='top',
    symbolType='stroke'
).configure_legend(symbolType='square', orient='top')

alt.Chart(...)

In [68]:
fireworks_chart['monthday'] = fireworks_chart['monthday'] + '/2020'

In [69]:
fireworks_chart.to_csv('output/fireworks_calls_datawrapper.csv', index=None)

In [70]:
this_year['monthday'] = this_year['monthday'] + '/2020'

In [71]:
this_year.to_csv('output/all_calls_datawrapper.csv', index=None)

---

## San Francisco

In [72]:
df_sf = pd.read_csv('/Users/mhustiles/data/data/LA/calls/san-francisco/san_francisco.csv', low_memory=False)

In [73]:
df_sf['date'] = pd.to_datetime(df_sf['date_time'], format='%Y-%m-%d %H:%M:%S')
df_sf['time'] = pd.to_datetime(df_sf['date_time'], format='%Y-%m-%d %H:%M:%S')

In [74]:
df_sf['monthday'] = df_sf['date'].dt.strftime('%m/%d')
df_sf['year'] = df_sf['date'].dt.strftime('%Y')

In [75]:
df_sf.head()

,crime_id,original_crime_type_name,report_date,call_date,offense_date,call_time,call_date_time,disposition,address,city,state,agency_id,address_type,common_location,date_time,date,time,monthday,year
0,193043877,Vandalism,10/31/2019,10/31/2019,10/31/2019,20:46,10/31/2019 08:46:00 PM,UTL,300 Block Of Toland St,San Francisco,CA,1,Premise Address,NaN,2019-10-31 08:46:00,2019-10-31 08:46:00,2019-10-31 08:46:00,10/31,2019
1,190020633,Trespasser,01/02/2019,01/02/2019,01/02/2019,07:58,01/02/2019 07:58:00 AM,GOA,300 Block Of Industrial St,San Francisco,CA,1,Premise Address,NaN,2019-01-02 07:58:00,2019-01-02 07:58:00,2019-01-02 07:58:00,01/02,2019
2,190214007,Passing Call,01/21/2019,01/21/2019,01/21/2019,23:07,01/21/2019 11:07:00 PM,HAN,Larkin St/golden Gate Av,San Francisco,CA,1,Intersection,NaN,2019-01-21 11:07:00,2019-01-21 11:07:00,2019-01-21 11:07:00,01/21,2019
3,190271719,22500e,01/27/2019,01/27/2019,01/27/2019,12:48,01/27/2019 12:48:00 PM,GOA,3400 Block Of Divisadero St,San Francisco,CA,1,Premise Address,NaN,2019-01-27 12:48:00,2019-01-27 12:48:00,2019-01-27 12:48:00,01/27,2019
4,190390109,Hot,02/08/2019,02/08/2019,02/08/2019,00:55,02/08/2019 12:55:00 AM,NOM,700 Block Of Corbett Av,San Francisco,CA,1,Premise Address,NaN,2019-02-08 12:55:00,2019-02-08 12:55:00,2019-02-08 12:55:00,02/08,2019


In [76]:
this_year_sf = df_sf[df_sf['year'] == '2020'].groupby(['monthday']).agg('size').reset_index(name='calls')

In [77]:
chart = alt.Chart((this_year_sf))\
    .mark_bar(size=2,opacity=1)\
    .encode(
    x=alt.X('monthday:T', title=' ', axis=alt.Axis(grid=False, tickCount=6, format='%B')),
    y=alt.Y('calls:Q', stack=None, title=' ', axis=alt.Axis(gridWidth=.6,\
     gridColor='#dddddd',offset=6,tickSize=0,domainOpacity=0,tickCount=4, format=''),\
           scale=alt.Scale(domain=(0, 2500)))
).properties(width=630, height=400,
     title='All service calls to San Francisco Police: 2020'
)

chart.configure_view(strokeOpacity=0).configure_legend(
    orient='top',
    symbolType='stroke'
).configure_legend(symbolType='square')

alt.Chart(...)

In [78]:
fireworks_melt.to_csv('output/fireworks_melt.csv')

In [79]:
this_year_sf.sort_values(by='monthday', ascending=False).head()

,monthday,calls
163,06/12,1531
162,06/11,1730
161,06/10,1759
160,06/09,1872
159,06/08,1737


---

## Sacramento

In [80]:
df_sac = pd.read_csv('/Users/mhustiles/data/data/LA/calls/sacramento/sacramento.csv', low_memory=False)

In [81]:
df_sac['date'] = pd.to_datetime(df_sac['date'], format='%Y-%m-%d')
df_sac['time'] = pd.to_datetime(df_sac['time'], format='%H:%M:%S')

In [82]:
df_sac['monthday'] = df_sac['date'].dt.strftime('%m/%d')
df_sac['year'] = df_sac['date'].dt.strftime('%Y')

In [83]:
this_year_sac = df_sac[df_sac['year'] == '2020'].groupby(['monthday']).agg('size').reset_index(name='calls')

In [84]:
chart = alt.Chart((this_year_sac))\
    .mark_bar(size=2,opacity=1)\
    .encode(
    x=alt.X('monthday:T', title=' ', axis=alt.Axis(grid=False, tickCount=6, format='%B')),
    y=alt.Y('calls:Q', stack=None, title=' ', axis=alt.Axis(gridWidth=.6,\
     gridColor='#dddddd',offset=6,tickSize=0,domainOpacity=0,tickCount=4, format=''),\
           scale=alt.Scale(domain=(0, 1500)))
).properties(width=630, height=400,
     title='All service calls to Sacramento Police: 2020'
)

chart.configure_view(strokeOpacity=0).configure_legend(
    orient='top',
    symbolType='stroke'
).configure_legend(symbolType='square')

alt.Chart(...)

In [85]:
this_year_sac.sort_values(by='monthday', ascending=False).head(5)

,monthday,calls
162,06/11,306
161,06/10,893
160,06/09,858
159,06/08,771
158,06/07,788


---

### Santa Monica

In [86]:
sm_df = pd.read_csv('/Users/mhustiles/data/data/LA/calls/santa_monica/santa_monica.csv', low_memory=False)

In [87]:
sm_df['date'] = pd.to_datetime(sm_df['received_time'], format='%Y-%m-%d %H:%M:%S')

In [88]:
sm_df['monthday'] = sm_df['date'].dt.strftime('%m/%d')
sm_df['year'] = sm_df['date'].dt.strftime('%Y')

In [89]:
this_year_sm = sm_df[sm_df['year'] == '2020'].groupby(['monthday']).agg('size').reset_index(name='calls')

In [90]:
this_year_sm.calls.mean()

265.09659090909093

In [91]:
chart = alt.Chart((this_year_sm))\
    .mark_bar(size=2,opacity=1)\
    .encode(
    x=alt.X('monthday:T', title=' ', axis=alt.Axis(grid=False, tickCount=6, format='%B')),
    y=alt.Y('calls:Q', stack=None, title=' ', axis=alt.Axis(gridWidth=.6,\
     gridColor='#dddddd',offset=6,tickSize=0,domainOpacity=0,tickCount=4, format=''),\
           scale=alt.Scale(domain=(0, 700)))
).properties(width=630, height=400,
     title='All service calls to this_year_sm Police: 2020'
)

chart.configure_view(strokeOpacity=0).configure_legend(
    orient='top',
    symbolType='stroke'
).configure_legend(symbolType='square')

alt.Chart(...)

In [92]:
this_year_sm['monthday'] = this_year_sm['monthday'] + '/2020'

In [93]:
this_year_sm.to_csv('output/santa_monica_calls_datawrapper.csv', index=None)

---

### San Diego

In [94]:
sd_df = pd.read_csv('/Users/mhustiles/data/data/LA/calls/san-diego/san_diego.csv', low_memory=False)

In [95]:
sd_df.head()

,incident_num,date_time,day_of_week,address_number_primary,address_dir_primary,address_road_primary,address_sfx_primary,address_dir_intersecting,address_road_intersecting,address_sfx_intersecting,call_type,disposition,beat,priority,date,time
0,E20010000001,2020-01-01 00:00:09,4,400,NaN,06TH,AVE,NaN,NaN,NaN,11-8,A,523,0,2020-01-01,00:00:09
1,E20010000002,2020-01-01 00:00:20,4,5000,NaN,UNIVERSITY,AVE,NaN,NaN,NaN,FD,K,826,2,2020-01-01,00:00:20
2,E20010000003,2020-01-01 00:00:21,4,800,NaN,SAWTELLE,AVE,NaN,NaN,NaN,AU1,W,434,1,2020-01-01,00:00:21
3,E20010000004,2020-01-01 00:00:32,4,5000,NaN,UNIVERSITY,AVE,NaN,NaN,NaN,FD,K,826,2,2020-01-01,00:00:32
4,E20010000005,2020-01-01 00:00:42,4,5200,NaN,CLAIREMONT MESA,BLV,NaN,NaN,NaN,415V,K,111,1,2020-01-01,00:00:42


In [96]:
sd_df['date'] = pd.to_datetime(sd_df['date_time'], format='%Y-%m-%d %H:%M:%S')

In [97]:
sd_df['monthday'] = sd_df['date'].dt.strftime('%m/%d')
sd_df['year'] = sd_df['date'].dt.strftime('%Y')

In [98]:
this_year_sd = sd_df[sd_df['year'] == '2020'].groupby(['monthday']).agg('size').reset_index(name='calls')

In [99]:
this_year_sd.loc[this_year_sd['monthday'] == '05/03', 'calls'] = 1557

In [100]:
chart = alt.Chart((this_year_sd.query('monthday < "06/16"')))\
    .mark_bar(size=2,opacity=1)\
    .encode(
    x=alt.X('monthday:T', title=' ', axis=alt.Axis(grid=False, tickCount=6, format='%B')),
    y=alt.Y('calls:Q', stack=None, title=' ', axis=alt.Axis(gridWidth=.6,\
     gridColor='#dddddd',offset=6,tickSize=0,domainOpacity=0,tickCount=4, format=''),\
           scale=alt.Scale(domain=(0, 2000)))
).properties(width=630, height=400,
     title='All service calls to San Diego Police: 2020'
)

chart.configure_view(strokeOpacity=0).configure_legend(
    orient='top',
    symbolType='stroke'
).configure_legend(symbolType='square')

alt.Chart(...)

---

### San Jose

In [101]:
sj_df = pd.read_csv('/Users/mhustiles/data/data/LA/calls/san-jose/san_jose.csv', low_memory=False)

In [102]:
sj_df['date'] = pd.to_datetime(sj_df['date'], format='%Y-%m-%d %H:%M:%S')

In [103]:
sj_df['monthday'] = sj_df['date'].dt.strftime('%m/%d')
sj_df['year'] = sj_df['date'].dt.strftime('%Y')

In [104]:
this_year_sj = sj_df[sj_df['year'] == '2020'].groupby(['monthday']).agg('size').reset_index(name='calls')

In [105]:
chart = alt.Chart((this_year_sj))\
    .mark_bar(size=2,opacity=1)\
    .encode(
    x=alt.X('monthday:T', title=' ', axis=alt.Axis(grid=False, tickCount=6, format='%B')),
    y=alt.Y('calls:Q', stack=None, title=' ', axis=alt.Axis(gridWidth=.6,\
     gridColor='#dddddd',offset=6,tickSize=0,domainOpacity=0,tickCount=4, format=''),\
           scale=alt.Scale(domain=(0, 1000)))
).properties(width=630, height=400,
     title='All service calls to San Jose Police: 2020'
)

chart.configure_view(strokeOpacity=0).configure_legend(
    orient='top',
    symbolType='stroke'
).configure_legend(symbolType='square')

alt.Chart(...)

In [106]:
sj_df.head()

,cdts,eid,start_date,call_number,priority,report_date,offense_date,offense_time,calltype_code,call_type,final_dispo_code,final_dispo,common_place_name,address,city,state,date,time,monthday,year
0,20200101002621PS,7981569,1/1/2020 12:00:00 AM,P200010004,2,1/1/2020 12:00:00 AM,1/1/2020 12:00:00 AM,00:03:08,1057,FIREARMS DISCHARGED,N,No report required; dispatch record only,NaN,[900]-[1000] FURLONG DR,San Jose,CA,2020-01-01,00:03:08,01/01,2020
1,20200101021606PS,7981674,1/1/2020 12:00:00 AM,P200010092,4,1/1/2020 12:00:00 AM,1/1/2020 12:00:00 AM,01:03:05,415,DISTURBANCE,CAN,Canceled,NaN,[0]-[100] PALM VALLEY BL,San Jose,CA,2020-01-01,01:03:05,01/01,2020
2,20200101021613PS,7981725,1/1/2020 12:00:00 AM,P200010134,5,1/1/2020 12:00:00 AM,1/1/2020 12:00:00 AM,01:50:32,647F,DRUNK IN PUBLIC,N,No report required; dispatch record only,NaN,S 1ST ST & E SAN SALVADOR ST,San Jose,CA,2020-01-01,01:50:32,01/01,2020
3,20200101021628PS,7981722,1/1/2020 12:00:00 AM,P200010131,2,1/1/2020 12:00:00 AM,1/1/2020 12:00:00 AM,01:49:04,415A,"DISTURBANCE, FIGHT",N,No report required; dispatch record only,NaN,NERDY AV & CAS DR,San Jose,CA,2020-01-01,01:49:04,01/01,2020
4,20200101021642PS,7981640,1/1/2020 12:00:00 AM,P200010064,2,1/1/2020 12:00:00 AM,1/1/2020 12:00:00 AM,00:43:04,911UNK,UNK TYPE 911 CALL,N,No report required; dispatch record only,NaN,[0]-[100] MERIDIAN AV,San Jose,CA,2020-01-01,00:43:04,01/01,2020
